# Minimal BAML Output Test

This notebook tests different methods of capturing BAML output.

In [ ]:
!pip install baml-py==0.202.0 pydantic

In [ ]:
import subprocess
import os
import sys

def baml_generate():
    result = subprocess.run(
        ["baml-cli", "generate"],
        check=True,
        capture_output=True,
        text=True
    )
    if result.stdout:
        print("[baml-cli generate]\n", result.stdout)
    if result.stderr:
        print("[baml-cli generate]\n", result.stderr)

def get_baml_client():
    # Set API key
    if 'OPENAI_API_KEY' not in os.environ:
        print("Warning: OPENAI_API_KEY not set")
    
    baml_generate()
    
    # Force delete all baml_client modules from sys.modules
    modules_to_delete = [key for key in sys.modules.keys() if key.startswith('baml_client')]
    for module in modules_to_delete:
        del sys.modules[module]
    
    # Now import fresh
    import baml_client
    return baml_client.sync_client.b

In [ ]:
!baml-cli init

In [ ]:
# Create a simple BAML function
baml_content = '''function DemoFunction {
  input: string
  output: string
}

impl<llm, DemoFunction> DemoFunctionImpl {
  prompt #"
    Say hello to {{input}}
  "#
}

client<llm> MyClient {
  provider openai
  options {
    model "gpt-4o-mini"
  }
}
'''

with open('baml_src/demo.baml', 'w') as f:
    f.write(baml_content)

print("Created demo.baml")

## Method 1: Direct Execution (Default)

In [ ]:
# Direct execution - BAML logs go to stderr
os.environ['BAML_LOG'] = 'info'
b = get_baml_client()
result = b.DemoFunction("World")
print(f"Result: {result}")

## Method 2: Capture with IPython

In [ ]:
from IPython.utils.capture import capture_output

os.environ['BAML_LOG'] = 'info'
print("Capturing output with IPython...")

with capture_output() as captured:
    b = get_baml_client()
    result = b.DemoFunction("IPython")

print(f"Result: {result}")
print("\n=== Captured stdout ===")
print(captured.stdout)
print("\n=== Captured stderr (BAML logs) ===")
print(captured.stderr)

## Method 3: Redirect stderr to stdout

In [ ]:
import contextlib

@contextlib.contextmanager
def redirect_stderr_to_stdout():
    old_stderr = sys.stderr
    sys.stderr = sys.stdout
    try:
        yield
    finally:
        sys.stderr = old_stderr

os.environ['BAML_LOG'] = 'info'
print("Redirecting stderr to stdout...")

with redirect_stderr_to_stdout():
    b = get_baml_client()
    result = b.DemoFunction("Redirect")

print(f"\nResult: {result}")

## Method 4: Cell Magic %%capture

In [ ]:
%%capture captured_output
os.environ['BAML_LOG'] = 'info'
b = get_baml_client()
result = b.DemoFunction("Cell Magic")
print(f"Result: {result}")

In [ ]:
# Display captured output
print("=== Captured stdout ===")
print(captured_output.stdout)
print("\n=== Captured stderr (BAML logs) ===")
print(captured_output.stderr)

## Method 5: Subprocess with combined output

In [ ]:
# Create a Python script that runs BAML
script_content = '''import os
os.environ['BAML_LOG'] = 'info'
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY', '')

import baml_client
b = baml_client.sync_client.b
result = b.DemoFunction("Subprocess")
print(f"Result: {result}")
'''

with open('test_baml_script.py', 'w') as f:
    f.write(script_content)

# Run as subprocess with combined output
result = subprocess.run(
    [sys.executable, 'test_baml_script.py'],
    capture_output=True,
    text=True,
    stderr=subprocess.STDOUT  # Combine stderr into stdout
)

print("=== Combined output ===")
print(result.stdout)

# Clean up
os.remove('test_baml_script.py')